In [1]:
# Enable autoreloading if import packages are changed
%load_ext autoreload
%autoreload 2

import src.sfm as sfm
import os
import torch
import json
import cv2
import os 
import config
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

PATH = os.getcwd()
print(PATH)
config.image_dir = os.path.join(PATH, 'data','')
print(config.image_dir)




def get_pc_and_poses(name, structure, color, rotation, motion):
    """
    load the json and ply files
    """
    # Generating rotation matrices from rotation vectors
    rotation_mat = [cv2.Rodrigues(rotation[i])[0] for i in range(len(rotation))]

    # Asserting the equal length of rotation matrices and motion vectors
    assert len(rotation_mat) == len(motion)

    # Combining rotation matrices and motion vectors to form extrinsic matrices
    extrinsic = np.concatenate((rotation_mat, motion), axis=2)

    # Converting extrinsic matrices into homogeneous coordinates
    extrinsic = np.array([np.row_stack((mat, [0, 0, 0, 1])) for mat in extrinsic]).tolist()

    # Getting a list of files in the image directory
    files = os.listdir(os.path.join(config.image_dir,name,'images',''))

    # Creating a dictionary with filenames as keys and extrinsic matrices as values
    dic = dict(zip(files, extrinsic))

    # Creating a dictionary containing intrinsic matrix and extrinsic matrices
    dic = {"intrinsics": config.K.tolist(), "extrinsics": dic}

    # Writing the dictionary to a JSON file
    with open(f"./data/{name}/estimated_camera_parameters.json", "w") as outfile:
        json.dump(dic, outfile)
    # print(f"pred_{name} file has been written")
    # Creating a point cloud object from the structure and color arrays
    pcd2 = o3d.geometry.PointCloud()
    pcd2.points = o3d.utility.Vector3dVector(structure)
    pcd2.colors = o3d.utility.Vector3dVector(color/255) #colors should be in the range 0-1
    o3d.io.write_point_cloud(f"./data/{name}/estimated_points.ply",pcd2,format ='xyzrgb')
    print(f"pred_{name} file has been written")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
d:\Gitte_Belly\Desktop\submission\project
d:\Gitte_Belly\Desktop\submission\project\data\


In [2]:

# Calling the 'main' function from the 'sfm' module, passing the 'device' variable as an argument
# Assigning the returned values (presumably structure, colors, rotations, and motions) to respective variables
structure_boot_stage1, color_boot_stage1, rotation_boot_stage1, motion_boot_stage1 = sfm.main('boot_stage1',device )
structure_boot_stage2, color_boot_stage2, rotation_boot_stage2, motion_boot_stage2 = sfm.main('boot_stage2',device )



extracting the correspondences
shape for one image: (1080, 1920, 3)


d:\Gitte_Belly\Desktop\submission\project\src\match.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  matches = torch.tensor(matches, dtype=torch.long, device=device)
d:\Gitte_Belly\Desktop\submission\project\src\match.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  distances = torch.tensor(distances, dtype=torch.float32, device=device)


initializing
reconstruct
filtering the outliers
3345
51
done
extracting the correspondences
shape for one image: (1080, 1920, 3)
initializing
reconstruct
filtering the outliers
7096
51
done


NameError: name 'obj_name' is not defined

In [5]:
get_pc_and_poses('boot_stage1', structure_boot_stage1, color_boot_stage1, rotation_boot_stage1, motion_boot_stage1)
get_pc_and_poses('boot_stage2', structure_boot_stage2, color_boot_stage2, rotation_boot_stage2, motion_boot_stage2)

pred_boot_stage1 file has been written
pred_boot_stage2 file has been written


## For milk and box

In [8]:
structure_box, color_box, rotation_box, motion_box     = sfm.main('box',device  )
structure_milk, color_milk, rotation_milk, motion_milk = sfm.main('milk',device  )

get_pc_and_poses('box', structure_box, color_box, rotation_box, motion_box)
get_pc_and_poses('milk', structure_milk, color_milk, rotation_milk, motion_milk)


extracting the correspondences
shape for one image: (1080, 1920, 3)
initializing
reconstruct
filtering the outliers
2557
76
done
extracting the correspondences
shape for one image: (1080, 1920, 3)
initializing
reconstruct
filtering the outliers
8568
50
done
pred_box.json file has been written
pred_box.ply file has been written
pred_milk.json file has been written
pred_milk.ply file has been written
